In [ ]:
import gym
import torch
import numpy as np
import sys
from torchvision.transforms import ToTensor, Lambda
from torch import nn
from my_model import Network
from utils import dataset, dataloader, buffer
import trainer
import random
import torch.onnx as onnx
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('RLresult/cartpole/pg')
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
env = gym.make('CartPole-v1') 
print("act", env.action_space)
print("obs", env.observation_space)

In [ ]:
HIDDEN_SIZE = 32
BATCH_SIZE = 10000
learning_rate = 0.01
CART_ACTION = 2
CAPACITY = 10000
TRAIN_ITER = CAPACITY*10/BATCH_SIZE
PARAM_PATH = 'PG_param/cartpolePG.pth'
PARAM_PATH_TEST = 'PG_param/cartpolePG_test.pth'
#print(Transition(2,1,2,3))
#Transition(state=2, action=1, next_state=2, reward=3)

In [ ]:
input_state_size = len(env.observation_space.sample())
updatedPG = Network.PG(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)


In [ ]:
updatedPG.load_state_dict(torch.load(PARAM_PATH_TEST))


In [ ]:
mybuffer = buffer.ForwardMem(updatedPG, env, "cart")

In [ ]:
mycartdata = dataset.EnvData(capacity = CAPACITY)

In [ ]:
%%time

mydataloader = dataloader.CustomDataLoader(mycartdata, batch_size=BATCH_SIZE)
mytrain = trainer.Train("cart", mydataloader, updatedDQN, updatedDQN)

In [ ]:

optimizer = torch.optim.SGD(updatedPG.parameters(), lr=learning_rate)
train_iter = 100
i = 0
while(i < train_iter):
    i = i + 1
    mybuffer.renewal_memory(CAPACITY, mycartdata)
    loss = mytrain.training(TRAIN_ITER, BATCH_SIZE, optimizer)
    writer.add_scalar("loss", loss, i)
    torch.save(updatedPG.state_dict(), PARAM_PATH_TEST)
    
    
env.close()